In [1]:
from recommand import *

100%|████████████████████████████████████████████████████████████████████████████████| 730/730 [02:04<00:00,  5.87it/s]


# 전체 데이터 유사도 추천

## 다음 url로 유사 기사 추천

In [226]:
def get_recommend_by_url(url, n, drange, press):
    import torch, time
    # from sklearn.metrics.pairwise import cosine_similarity
    # 다음 뉴스인지 확인
    if not url.startswith('https://v.daum.net/v/'):
        return False
    
    ## 입력 처리
    # 본문 스크랩
    content = content_only_scraper(url)
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    start = time.time()
    # tfidf 연산
    tfidf_content = tfidf_vec.transform([token])
    print(f"tfidf 계산 시간 : {time.time()-start}")
    ## 조회 한정
    # 기간 & 언론사
    start = time.time()
    data_filtered_idx = data_all[(data_all['press']==press) & (data_all['date'].isin(pd.date_range(drange[0], drange[1])))].index
    tfidf_dtm_filtered = tfidf_dtm[data_filtered_idx]
    cos_sim_res = cos_similarity(tfidf_content, tfidf_dtm_filtered)
    print(f"코사인 계산 시간 : {time.time()-start}")

    # 유사도 정렬, 추출
    sim_scores_n = torch.topk(torch.Tensor(cos_sim_res), k=n)
    
    # 해당하는 기사 가져오기
    article_idx = data_filtered_idx[sim_scores_n[1][0]]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim.item(), 4)*100 for sim in sim_scores_n[0][0]]
    
    return rst[['press', 'similarity', 'title', 'date', 'link']]

# cos_sim 비교

### sklearn의 cosine_similarity

In [227]:
def cos_similarity(content, dtm_filtered):
    from sklearn.metrics.pairwise import cosine_similarity
    return cosine_similarity(content, dtm_filtered)

In [228]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10, ("2022-01-01", "2022-12-31"), "뉴시스")

tfidf 계산 시간 : 0.01927661895751953
코사인 계산 시간 : 0.7079620361328125
CPU times: total: 1.12 s
Wall time: 1.2 s


,press,similarity,title,date,link
1358639,뉴시스,48.96,'입시비리'·감찰무마' 조국 재판 오늘 결심…3년 만에 마무리 수순,2022-12-02,https://v.daum.net/v/20221202060032795
1357681,뉴시스,48.84,"'자녀 입시비리' 조국 전 장관, 1심 결심공판 출석 [뉴시스Pic]",2022-12-02,https://v.daum.net/v/20221202141500738
1330596,뉴시스,48.03,"檢 ""입시비리 중대범죄"", 정경심 측 ""부당 기소""…징역 2년 구형",2022-11-18,https://v.daum.net/v/20221118115238952
1346082,뉴시스,47.49,"조국 입시비리 등 의혹, 이번주 결심…이르면 올해 1심 결론",2022-11-27,https://v.daum.net/v/20221127095726043
1357622,뉴시스,47.43,"검찰, 조국 '자녀 입시비리·감찰무마 혐의' 징역 5년 구형",2022-12-02,https://v.daum.net/v/20221202143703564
1357589,뉴시스,46.34,"조국 징역 5년 구형…검찰 ""명백한 사실조차 인정 안해""(종합)",2022-12-02,https://v.daum.net/v/20221202145411164
1330731,뉴시스,44.06,"檢, 정경심 '아들 입시비리 혐의' 징역 2년 구형…""중대범죄""",2022-11-18,https://v.daum.net/v/20221118110927242
1316908,뉴시스,43.58,"檢, 백원우·박형철 '감찰무마 혐의' 실형 구형…조국 ""文 청와대 모독""(종합)",2022-11-11,https://v.daum.net/v/20221111122348143
1357047,뉴시스,43.05,"'징역 5년' 구형된 조국 ""검찰권 앞에 무력…하루하루 생지옥""",2022-12-02,https://v.daum.net/v/20221202185512175
1317109,뉴시스,42.60,"'유재수 감찰무마 혐의' 백원우·박형철, 징역 2년·1년6월 구형",2022-11-11,https://v.daum.net/v/20221111112240978


### torch의 CosineSimilarity

In [160]:
def convert_sparse_matrix_to_sparse_tensor(X):
    import numpy as np
    import tensorflow as tf
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [165]:
def cos_similarity(content, dtm_filtered):
    from torch.nn import CosineSimilarity
    content = convert_sparse_matrix_to_sparse_tensor(content)
    dtm_filtered = convert_sparse_matrix_to_sparse_tensor(dtm_filtered)
    cos = CosineSimilarity(dim=1)
    return cos(content, dtm_filtered)

In [166]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10, ("2021-01-01", "2022-12-31"), "뉴시스")

TypeError: cosine_similarity(): argument 'x1' (position 1) must be Tensor, not SparseTensor

### scipy의 cosine

In [171]:
def cos_similarity(content, dtm_filtered):
    from scipy.spatial.distance import cosine
    return cosine(content, dtm_filtered)

In [172]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10, ("2021-01-01", "2022-12-31"), "뉴시스")

ValueError: Input vector should be 1-D.

### numpy로 직접 구현

In [62]:
def cos_similarity(content, dtm_filtered):
    import sklearn.preprocessing as pp
    normed_mat = pp.normalize(content.tocsc(), axis=0)
    return normed_mat.T*dtm_filtered

In [63]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10, ("2021-01-01", "2022-12-31"), "뉴시스")

ValueError: dimension mismatch